In [1]:
from orm_loader.registry import ModelRegistry, ValidationRunner, always_on_validators
from orm_loader.helpers import configure_logging, bootstrap
from omop_alchemy.cdm.specification import TABLE_LEVEL_CSV, FIELD_LEVEL_CSV
from omop_alchemy import get_engine_name, load_environment, TEST_PATH, ROOT_PATH
import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker

configure_logging()
load_environment()

engine_string = get_engine_name()
engine = sa.create_engine(engine_string, future=True, echo=False)
registry = ModelRegistry(model_name='CDM', model_version="5.4")

registry.load_table_specs(
    table_csv=TABLE_LEVEL_CSV,
    field_csv=FIELD_LEVEL_CSV,
)

registry.discover_models("omop_alchemy.cdm.model")
bootstrap(engine, create=True)

2026-01-12 10:07:02,020 | INFO     | sql_loader.omop_alchemy.config | Environment variables loaded from .env file
2026-01-12 10:07:02,020 | INFO     | sql_loader.omop_alchemy.config | Database engine configured


In [2]:
list(registry.known_tables())[:5]

['care_site',
 'fact_relationship',
 'cost',
 'condition_occurrence',
 'payer_plan_period']

In [3]:
validators = always_on_validators()
runner = ValidationRunner(
    validators=validators,
    fail_fast=False,
)
report = runner.run(registry)


In [4]:
# report = registry.validate(engine=engine, check_domain_semantics=True)

In [5]:
print(report.summary())

MODEL v5.4: 0 error(s), 27 warning(s), 8 info


In [6]:
if not report.is_valid():
    print(report.render_text_report())


📦 cdm_source
  ⚠️ PRIMARY_KEY_NOT_DECLARED_IN_SPEC (field: cdm_source_name) Hint: ORM primary key not marked as primary key in specification

📦 cohort
  ⚠️ PRIMARY_KEY_NOT_DECLARED_IN_SPEC (field: cohort_definition_id) Hint: ORM primary key not marked as primary key in specification
  ⚠️ PRIMARY_KEY_NOT_DECLARED_IN_SPEC (field: subject_id) Hint: ORM primary key not marked as primary key in specification
  ⚠️ COMPOSITE_PRIMARY_KEY Hint: Composite primary key detected

📦 cohort_definition
  ⚠️ PRIMARY_KEY_NOT_DECLARED_IN_SPEC (field: cohort_definition_id) Hint: ORM primary key not marked as primary key in specification

📦 concept_ancestor
  ⚠️ PRIMARY_KEY_NOT_DECLARED_IN_SPEC (field: ancestor_concept_id) Hint: ORM primary key not marked as primary key in specification
  ⚠️ PRIMARY_KEY_NOT_DECLARED_IN_SPEC (field: descendant_concept_id) Hint: ORM primary key not marked as primary key in specification
  ⚠️ COMPOSITE_PRIMARY_KEY Hint: Composite primary key detected

📦 concept_relationship


In [ ]:
for table, spec in registry._table_specs.items():
    print(f"{table}: {spec.is_required}")